The code for creating the baseline comes from [this Jupyter notebook](https://github.com/marshmellow77/text-summarisation-project/blob/main/1_baseline_model.ipynb).

In [ ]:
# !pip install datasets rouge_score

In [ ]:
import re
import pandas as pd
import datasets

In [ ]:
data = datasets.load_dataset("cnn_dailymail", "3.0.0")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
train_data, test_data, val_data = data["train"], data["test"], data["validation"]
train_df, test_df, val_df = train_data.to_pandas(), test_data.to_pandas(), val_data.to_pandas()

Create the baselines using the first, second, and third sentences for each row in the dataframe.

In [ ]:
display(train_df.head(5))
display(test_df.head(5))

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


,article,highlights,id
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f


In [ ]:
df = pd.concat([train_df, val_df, test_df], ignore_index = True)
df = df.astype(str)
df["article"] = df["article"].str[1:-1]
df["highlights"] = df["highlights"].str[1:-1]
df["id"] = df["id"].str[1:-1]

In [ ]:
ref_summaries = df["highlights"]
display(ref_summaries.head(5))

0    arry Potter star Daniel Radcliffe gets £20M fo...
1    entally ill inmates in Miami are housed on the...
2    EW: "I thought I was going to die," driver say...
3    ive small polyps found during procedure; "none...
4    EW: NFL chief, Atlanta Falcons owner critical ...
Name: highlights, dtype: object

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
for i in range(3):
  candidate_summaries = list(df["article"].apply(lambda x: " ".join(re.split(r"(?<=[.:;])\s", x)[:i+1])))
  print(f"Scores: {calc_rouge_scores(candidate_summaries, ref_summaries)}")

Scores: {'rouge1': 16.4, 'rouge2': 5.5, 'rougeL': 11.4, 'rougeLsum': 14.2}
Scores: {'rouge1': 22.9, 'rouge2': 8.7, 'rougeL': 14.6, 'rougeLsum': 19.1}
Scores: {'rouge1': 26.7, 'rouge2': 10.6, 'rougeL': 17.0, 'rougeLsum': 22.2}
